<a href="https://colab.research.google.com/github/chinmay002/Pytorch/blob/main/LSTM_and_simple_classificaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
import re,json,string,os
import collections
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [2]:
n_features= 2
hidden_dim =2

torch.manual_seed(17)
lstm_cell = nn.LSTMCell(input_size = n_features,hidden_size = hidden_dim)
lstm_state = lstm_cell.state_dict()
lstm_state

OrderedDict([('weight_ih',
              tensor([[-0.0930,  0.0497],
                      [ 0.4670, -0.5319],
                      [-0.6656,  0.0699],
                      [-0.1662,  0.0654],
                      [-0.0449, -0.6828],
                      [-0.6769, -0.1889],
                      [-0.4167, -0.4352],
                      [-0.2060, -0.3989]])),
             ('weight_hh',
              tensor([[-0.7070, -0.5083],
                      [ 0.1418,  0.0930],
                      [-0.5729, -0.5700],
                      [-0.1818, -0.6691],
                      [-0.4316,  0.4019],
                      [ 0.1222, -0.4647],
                      [-0.5578,  0.4493],
                      [-0.6800,  0.4422]])),
             ('bias_ih',
              tensor([-0.3559, -0.0279,  0.6553,  0.2918,  0.4007,  0.3262, -0.0778, -0.3002])),
             ('bias_hh',
              tensor([-0.3991, -0.3200,  0.3483, -0.2604, -0.1582,  0.5558,  0.5761, -0.3919]))])

In [3]:
Wx,bx = lstm_state['weight_ih'],lstm_state['bias_ih']
Wh,bh = lstm_state['weight_hh'],lstm_state['bias_hh']

print(Wx.shape,bx.shape)

torch.Size([8, 2]) torch.Size([8])


In [10]:
Wxi,Wxf,Wxg,Wxo = Wx.split(hidden_dim) 
bxi,bxf,bxg,bxo = bx.split(hidden_dim) 
Whi,Whf,Whg,Who = Wh.split(hidden_dim) 
bhi,bhf,bhg,bho = bh.split(hidden_dim) 

In [11]:
print(Wxi.shape,Wxf.shape,Wxg.shape,Wxo.shape)

torch.Size([2, 2]) torch.Size([2, 2]) torch.Size([2, 2]) torch.Size([2, 2])


In [13]:
def linear_layer(Wx,bx,Wh,bh):
  hidden_dim , n_feat = Wx.size()#(2,2)
  lin_input = nn.Linear(n_feat,hidden_dim)
  lin_input.load_state_dict({'weight':Wx,'bias':bx})
  lin_hidden = nn.Linear(hidden_dim,hidden_dim)
  lin_hidden.load_state_dict({'weight':Wh,'bias':bh})

  return lin_hidden,lin_input

In [17]:
i_hidden,i_input = linear_layer(Wxi,bxi,Whi,bhi)
f_hidden,f_input = linear_layer(Wxf,bxf,Whf,bhf)
o_hidden,o_input = linear_layer(Wxo,bxo,Who,bho)
#these are gates, we need to create functions for the,/

#for g , since its an RNN cell, we load_dict to create 
g_cell = nn.RNNCell(2,2)
g_cell.load_state_dict({'weight_ih':Wxg,'bias_ih':bxg,'weight_hh':Whg,'bias_hh':bhg})

<All keys matched successfully>

In [15]:
def forget_gate(h,x):
  thf = f_hidden(h)
  txf = f_input(x)
  f = torch.sigmoid(thf+txf)
  return f

def output_gate(h,x):
  tho = o_hidden(h)
  txo = o_input(x)
  o = torch.sigmoid(tho+txo)
  return o

def input_gate(h,x):
  thi = i_hidden(h)
  txi = i_input(x)
  i = torch.sigmoid(thi+txi)
  return i    

In [23]:
#inital hidden vlaues
initial_hidden = torch.zeros(1,hidden_dim)
initial_cell = torch.zeros(1,hidden_dim)

X = torch.as_tensor([[ 1.0349,  0.9661]])

g = g_cell(X)
print(g)
i = input_gate(initial_hidden,X)
gated_input = g*i
print(gated_input)

f = forget_gate(initial_hidden,X)
print(f)
c_ = (f*initial_cell) + (gated_input)
print(c_)


o = output_gate(initial_hidden,X)
print(o)
h_ = o*torch.tanh(c_)
print(h_)

tensor([[-0.4330, -0.0010]], grad_fn=<TanhBackward0>)
tensor([[-0.1339, -0.0004]], grad_fn=<MulBackward0>)
tensor([[0.5944, 0.4807]], grad_fn=<SigmoidBackward0>)
tensor([[-0.1339, -0.0004]], grad_fn=<AddBackward0>)
tensor([[0.4126, 0.2157]], grad_fn=<SigmoidBackward0>)
tensor([[-5.4934e-02, -8.4665e-05]], grad_fn=<MulBackward0>)


In [24]:
lstm_cell(X)

(tensor([[-5.4934e-02, -8.4665e-05]], grad_fn=<MulBackward0>),
 tensor([[-0.1339, -0.0004]], grad_fn=<AddBackward0>))

**Variable_Length Sequences**

In [26]:
X = torch.as_tensor([[ 1.0349,  0.9661],
        [ 0.8055, -0.9169],
        [-0.8251, -0.9499],
        [-0.8670,  0.9342]])
X.shape

torch.Size([4, 2])

In [28]:
x0 = torch.randn((4, 2))
x1 = torch.randn((2, 2))
x2 = torch.randn((3, 2))

In [29]:
print(x0.shape,x1.shape,x2.shape)

torch.Size([4, 2]) torch.Size([2, 2]) torch.Size([3, 2])


In [33]:
#since they have different shape, we cannot combine them to make a list since.
all_seqs = [x0,x1,x2]
torch.as_tensor(all_seqs)

ValueError: ignored

In [37]:
seq_tensor = [torch.as_tensor(seq).float() for seq in all_seqs] 
padded = nn.utils.rnn.pad_sequence(seq_tensor,batch_first=True)
padded.shape

torch.Size([3, 4, 2])

In [39]:
rnn = nn.RNN(2,2,batch_first=True)
output,hidden = rnn(padded)
output,hidden

(tensor([[[ 0.0983,  0.5139],
          [-0.2063,  0.7669],
          [ 0.1308,  0.6461],
          [-0.4824,  0.8739]],
 
         [[ 0.4446,  0.3512],
          [ 0.1001,  0.5156],
          [ 0.0751,  0.5899],
          [ 0.0532,  0.5817]],
 
         [[ 0.1895,  0.8576],
          [-0.4430,  0.8226],
          [ 0.3189,  0.6641],
          [ 0.1922,  0.4961]]], grad_fn=<TransposeBackward1>),
 tensor([[[-0.4824,  0.8739],
          [ 0.0532,  0.5817],
          [ 0.1922,  0.4961]]], grad_fn=<StackBackward0>))

In [43]:
output[-1][-1] == hidden[-1][-1]

tensor([True, True])

In [44]:
import seaborn as sns
import pandas as pd
import numpy as np
import os
import json
import re
import torch.nn as nn
import torchvision
from torch.optim import optimizer
from torch.utils.data import Dataset, dataloader
import torch.nn.functional as f

In [48]:
df = pd.read_csv('/content/women_sentiment.csv').iloc[:,1:]
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [52]:
df= df[['Review Text',	'Recommended IND']]
df.columns = ['text','target']

In [53]:
df.head()

,text,target
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [59]:

df = df.dropna()
df.isnull().sum()

text      0
target    0
dtype: int64

In [60]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2)
vectorizer = CountVectorizer(lowercase=True)
vectorizer.fit(df['text'])
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
print(X_train_vec.shape)
print(X_test_vec.shape)

(18112, 14145)
(4529, 14145)


In [75]:
X_train_tensor = torch.from_numpy(X_train_vec.todense()).float()
X_test_tensor = torch.from_numpy(X_test_vec.todense()).float()
y_train_tensor = torch.from_numpy(np.array(y_train)).float()
y_test_tensor = torch.from_numpy(np.array(y_test)).float()

In [76]:
X_train_tensor.shape,X_test_tensor.shape,y_train_tensor.shape

(torch.Size([18112, 14145]), torch.Size([4529, 14145]), torch.Size([18112]))

In [83]:
class Network(torch.nn.Module):
  def __init__(self,vocab_size,out_classes):
        super().__init__()
        self.linear = torch.nn.Linear(vocab_size,out_classes)
  def forward(self,x):
        return self.linear(x) 


vocab_size = X_train_tensor.shape[1]
output_dim = 1

model = Network(vocab_size,output_dim)
pred = model(X_train_tensor[1]) #just get pred for one inp

In [84]:
pred

tensor([-0.1100], grad_fn=<AddBackward0>)

We’ll use DataLoader class for batching but it requires TensorDataset

In [86]:
from torch.utils.data import Dataset, TensorDataset

train_data = TensorDataset(X_train_tensor,y_train_tensor)  #return a list of tuple(feat1,tar1),(feat2,tar2)
train_data.tensors

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([0., 1., 1.,  ..., 1., 1., 1.]))

In [87]:
train_data[1]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor(1.))

In [94]:
train_loader = DataLoader(train_data,batch_size=16,shuffle=True)

next(iter(train_loader))

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1.])]

In [109]:
class Network(torch.nn.Module):
  def __init__(self,vocab_size,hidden_units,num_classes): 
    super().__init__()
     #First fully connected layer
    self.fc1 = torch.nn.Linear(vocab_size,hidden_units)
      #Second fully connected layer
    self.fc2 = torch.nn.Linear(hidden_units,num_classes)
      #Final output of sigmoid function      
    self.output = torch.nn.Sigmoid()
  def forward(self,x):
    fc1 = self.fc1(x)
    fc2 = self.fc2(fc1)
    output = self.output(fc2)  #gives shape [16,1]--batch size and yBatch is of size 16
    #print(f'output.shape is {output.squeeze(dim=1).shape}')
    return output.squeeze(dim=1)

1. Forward pass

1.   Calculate the loss
2.   Zero gradients
3.   backpropogation
4.   update optimizer










In [111]:
batch_size = 32
NUM_EPOCHS = 2
VOCAB_SIZE = 14145
HIDDEN_UNITS = 3
OUT_CLASSES = 1
LEARNING_RATE = 0.001

#Initialize model
model = Network(VOCAB_SIZE,HIDDEN_UNITS,OUT_CLASSES)
print(model)
#Initialize optimizer
optimizer =torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
#Initialize loss function
loss_fun = torch.nn.BCELoss()


for epoch in range(NUM_EPOCHS):
  for x_batch, y_batch in train_loader:
    model.train()
    y_pred = model(x_batch)

    y_loss = loss_fun(y_pred,y_batch)

    optimizer.zero_grad()
    y_loss.backward()
    optimizer.step()
  print('After {} epoch training loss is {}'.format(epoch,y_loss.item()))

Network(
  (fc1): Linear(in_features=14145, out_features=3, bias=True)
  (fc2): Linear(in_features=3, out_features=1, bias=True)
  (output): Sigmoid()
)
After 0 epoch training loss is 0.5943374037742615
After 1 epoch training loss is 0.5806816816329956
